In [1]:
# Import packages
import pyspark
import dxpy
import dxdata

In [2]:
# Spark initialization (Done only once; do not rerun this cell unless you select Kernel -> Restart kernel).
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/cluster/dnax/jars/dnanexus-api-0.1.0-SNAPSHOT-jar-with-dependencies.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/cluster/spark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2023-10-03 21:05:39.768 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2023-10-03 21:05:40.769 WARN  Utils:69 - Service 'org.apache.spark.network.netty.NettyBlockTransferService' could not bind on port 43000. Attempting port 43001.
2023-10-03 21:05:41.002 WARN  MetricsReporter:84 - No metrics configured for reporting
2023-10-03 21:05:41.004 WARN  LineProtoUsageReporter:48 - Telegraf configurations: url [metrics.push.telegraf.hostport], user [metrics.push.telegraf.user] or password [metrics.push.telegraf.password] missing.
2023-10-03 21:05:41.004 WARN  MetricsReporter:117 - metrics.scraping.httpserver.port


In [3]:
# Automatically discover dispensed database name and dataset id
dispensed_database = dxpy.find_one_data_object(
    classname='database', 
    name='app*', 
    folder='/', 
    name_mode='glob', 
    describe=True)
dispensed_database_name = dispensed_database['describe']['name']

dispensed_dataset = dxpy.find_one_data_object(
    typename='Dataset', 
    name='app*.dataset', 
    folder='/', 
    name_mode='glob')
dispensed_dataset_id = dispensed_dataset['id']

In [4]:
dataset = dxdata.load_dataset(id=dispensed_dataset_id)

In [5]:
participant = dataset['participant']

In [6]:
field_name_dict = {
    'sample_names': 'eid',
    'sex': 'p31',
    'bmi0': 'p21001_i0',
    'bmi1': 'p21001_i1',
    'bmi2': 'p21001_i2',
    'genetic_kinship_to_other_participants': 'p22021',
    'genetic_ethnic_grouping': 'p22006',
    'genetic_sex': 'p22001',
    'year_of_birth': 'p34',
    'menopause0': 'p2724_i0',
    'menopause1': 'p2724_i1',
    'menopause2': 'p2724_i2',
    'ethnic_background0': 'p21000_i0',
    'ethnic_background1': 'p21000_i1',
    'ethnic_background2': 'p21000_i2',
    'bmi_prs': 'p26216',
    'age_assessment0': 'p21003_i0',
    'age_assessment1': 'p21003_i1',
    'age_assessment2': 'p21003_i2'
}

for idx in range(1, 41):
    field_name_dict[f'genetic_pca{idx}'] = f'p22009_a{idx}'

In [7]:

field_names = list(field_name_dict.values())



In [8]:
df = participant.retrieve_fields(names=field_names, engine=dxdata.connect(), coding_values="replace")

2023-10-03 21:05:58.839 WARN  ShellBasedUnixGroupsMapping:210 - unable to return groups for user Vx9Zg8q9PVYZ9PxjzkBK3bBZVJ36vQPq0yzKY3YQ__project-GQpgZf8JX0KKbFBGK9yff4Zg
PartialGroupNameException The user name 'Vx9Zg8q9PVYZ9PxjzkBK3bBZVJ36vQPq0yzKY3YQ__project-GQpgZf8JX0KKbFBGK9yff4Zg' is not found. id: ‘Vx9Zg8q9PVYZ9PxjzkBK3bBZVJ36vQPq0yzKY3YQ__project-GQpgZf8JX0KKbFBGK9yff4Zg’: no such user
id: ‘Vx9Zg8q9PVYZ9PxjzkBK3bBZVJ36vQPq0yzKY3YQ__project-GQpgZf8JX0KKbFBGK9yff4Zg’: no such user

	at org.apache.hadoop.security.ShellBasedUnixGroupsMapping.resolvePartialGroupNames(ShellBasedUnixGroupsMapping.java:294)
	at org.apache.hadoop.security.ShellBasedUnixGroupsMapping.getUnixGroups(ShellBasedUnixGroupsMapping.java:207)
	at org.apache.hadoop.security.ShellBasedUnixGroupsMapping.getGroups(ShellBasedUnixGroupsMapping.java:97)
	at org.apache.hadoop.security.JniBasedUnixGroupsMappingWithFallback.getGroups(JniBasedUnixGroupsMappingWithFallback.java:51)
	at org.apache.hadoop.security.Groups$Gro

In [9]:
# See the first five entries as a Pandas DataFrame:
df.limit(5).toPandas()


2023-10-03 21:06:10.259 WARN  package:69 - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,eid,p31,p21001_i0,p21001_i1,p21001_i2,p22021,p22006,p22001,p34,p2724_i0,...,p22009_a31,p22009_a32,p22009_a33,p22009_a34,p22009_a35,p22009_a36,p22009_a37,p22009_a38,p22009_a39,p22009_a40
0,3351884,Female,34.1036,NaN,NaN,No kinship found,Caucasian,Female,1961,Not sure - other reason,...,-7.316860,1.283350,3.97141,1.290280,1.06482,6.63408,3.03357,-1.999740,5.446890,-4.35153
1,4366755,Female,37.9464,NaN,NaN,No kinship found,Caucasian,Female,1960,No,...,1.745990,0.585117,1.51662,0.994568,-1.40280,3.86994,3.59829,1.325750,-0.374176,9.42301
2,1270557,Female,25.1205,NaN,NaN,No kinship found,Caucasian,Female,1943,Yes,...,-1.936160,2.739070,-4.62403,-5.848920,3.94096,3.08413,1.04149,-0.374862,-2.457350,-3.94002
3,3439885,Male,31.6320,NaN,NaN,No kinship found,Caucasian,Male,1945,None,...,2.886250,1.225440,2.22903,6.779000,5.59474,3.49702,-1.54641,1.972960,-1.609130,-1.94949
4,4854010,Female,28.4083,NaN,NaN,At least one relative identified,Caucasian,Female,1950,Yes,...,0.493258,3.893070,-5.88067,0.579989,2.76152,-1.07788,6.20570,-5.159830,-2.729040,-2.20650


In [10]:
def upload_file_to_project(filename, proj_dir):
    dxpy.upload_local_file(filename, folder=proj_dir, parents=True)
    print(f"*********{filename} uploaded!!*********")
    return

In [11]:
pandas_df = df.toPandas()

In [12]:
pandas_df = pandas_df.rename(columns={v:k for k,v in field_name_dict.items()})

In [13]:
start = 0
final = len(pandas_df)
block = 0
proj_dir = f"/phenotype_processing/bmi_info/"
nrows = 50000

for i in range(10):
    end = min(start+nrows, final)
    table = pandas_df.iloc[start:end, :]
    filename = f"bmi_block{block}.csv.gz"
    table.to_csv(filename, index=False)
    upload_file_to_project(filename, proj_dir)
    start += nrows
    block += 1
    

*********bmi_block0.csv.gz uploaded!!*********
*********bmi_block1.csv.gz uploaded!!*********
*********bmi_block2.csv.gz uploaded!!*********
*********bmi_block3.csv.gz uploaded!!*********
*********bmi_block4.csv.gz uploaded!!*********
*********bmi_block5.csv.gz uploaded!!*********
*********bmi_block6.csv.gz uploaded!!*********
*********bmi_block7.csv.gz uploaded!!*********
*********bmi_block8.csv.gz uploaded!!*********
*********bmi_block9.csv.gz uploaded!!*********


# Resources

1. https://dnanexus.gitbook.io/uk-biobank-rap/working-on-the-research-analysis-platform/using-spark-to-analyze-tabular-data
2. https://github.com/dnanexus/OpenBio/blob/master/UKB_notebooks/ukb-rap-pheno-basic.ipynb